<a href="https://colab.research.google.com/github/marinathomas/SentimentAnalysisHN/blob/master/HN_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.cloud import bigquery
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
credential_path = "/content/gdrive/Shared drives/HackerNews:SentimentAnalysis/BigData-HackerNews-77d9fa1b02c1.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

1. Load the BigQuery client
2. Get a reference to HackerNews dataset
3. Load the data set

In [0]:
client = bigquery.Client()
hn_dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')
hn_dset = client.get_dataset(hn_dataset_ref)

In [4]:
query = """
select  *
from `bigquery-public-data.hacker_news.full` 
where type = 'comment' and  EXTRACT(YEAR FROM timestamp)=2017 and 
parent in (select id from `bigquery-public-data.hacker_news.full` where REGEXP_CONTAINS(title, r"(S|s)how HN"))
order by parent;
"""
query_job = client.query(query)
iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
comments = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
comments.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"<a href=""https:&#x2F;&#x2F;www.youtube.com&#x2...",None,i-make-robots,None,1483490034,2017-01-04 00:33:54+00:00,comment,13314650,13254029,None,None,None
1,None,None,Good idea. I will add the feature soon in upco...,None,palerdot,None,1483242597,2017-01-01 03:49:57+00:00,comment,13294635,13283785,None,None,None
2,None,None,Doesn&#x27;t SoundCloud offer this already?,None,egfx,None,1483292904,2017-01-01 17:48:24+00:00,comment,13296488,13285641,None,None,None
3,None,None,This is really awesome. Did you learn to built...,None,rohmanhakim,None,1483358902,2017-01-02 12:08:22+00:00,comment,13300687,13290312,None,None,None
4,None,None,This is cool and I&#x27;m tempted. However I ...,None,FullyFunctional,None,1484015060,2017-01-10 02:24:20+00:00,comment,13362016,13290312,None,None,None


In [30]:
query = """
select  *
from `bigquery-public-data.hacker_news.full` 
where type = 'comment' and  
parent in (select id from `bigquery-public-data.hacker_news.full` where REGEXP_CONTAINS(title, r"(S|s)how HN") and descendants > 550)
order by parent, id asc;


"""

query_job = client.query(query)
iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
max_desc_comments = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
max_desc_comments.fillna(0, inplace=True)
# Look at the first 10 top scores
max_desc_comments.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,0,0,"Looks neat. As an aside, why is it named after...",0,tabeth,0,1496759365,2017-06-06 14:29:25+00:00,comment,14497440,14497295,0,0,0
1,0,0,Founder here. Here to answer any questions you...,0,robinhood,0,1496759534,2017-06-06 14:32:14+00:00,comment,14497461,14497295,0,0,0
2,0,0,I like it! I definitely could use something li...,0,fil_a_del_fee_a,0,1496759543,2017-06-06 14:32:23+00:00,comment,14497464,14497295,0,0,0
3,0,0,Thanks for making this open-source. It seems t...,0,positivecomment,0,1496759932,2017-06-06 14:38:52+00:00,comment,14497536,14497295,0,0,0
4,0,0,Looks interesting. Definitely something that c...,0,TheWindUpPirate,0,1496760006,2017-06-06 14:40:06+00:00,comment,14497549,14497295,0,0,0


In [7]:
import paralleldots

ModuleNotFoundError: ignored

In [8]:
!pip install paralleldots

In [0]:
import paralleldots
paralleldots.set_api_key( "d7eDaY9Dgq4C6mLYSLuUqFTZEJJUZ4fwhlMqBgcbClY" )

In [16]:
for index,row in max_desc_comments.iterrows():
    print(row['text'])
    print(paralleldots.sentiment(row['text']))

CRM? No... PRM... Personal Relationship Management. :)
{'sentiment': {'negative': 0.209, 'neutral': 0.648, 'positive': 0.143}}
Tried to signup with facebook and got the error: &quot;The redirect_uri URL must be absolute&quot;
{'sentiment': {'negative': 0.724, 'neutral': 0.232, 'positive': 0.043}}
&gt;<i>Here are everything you can do with Monica</i><p>&gt;<i>Manage significant others</i><p>I really hope the writers of <i>Silicon Valley</i> are taking notes right now.
{'sentiment': {'negative': 0.11, 'neutral': 0.599, 'positive': 0.291}}
Great work! I had a similar idea and built <a href="https:&#x2F;&#x2F;socialite.ooo" rel="nofollow">https:&#x2F;&#x2F;socialite.ooo</a>. It&#x27;s been a bit neglected as I was finishing up grad school. It has a heavy focus on events and locations to tie everything together.
{'sentiment': {'negative': 0.149, 'neutral': 0.305, 'positive': 0.546}}
IMHO: You want to pivot this product, now, to compete with ourfamilywizard.com. OFW is a great concept but th

In [17]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.9MB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [31]:
for index,row in max_desc_comments.iterrows():
    sentence = row['text']
    score = analyser.polarity_scores(str(sentence))
    print("{:-<40} {}".format(sentence, str(score)))


Looks neat. As an aside, why is it named after a woman? I&#x27;m genuinely curious. {'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'compound': 0.6486}
Founder here. Here to answer any questions you might have. The site is not perfect, it&#x27;s not mobile optimized, there are probably bugs, there is a gazillion features missing, no APIs but it&#x27;s a labour of love, open-source and I hope it will help people other than me. I want to grow this product but I need to know what you need, people.
Edit: sorry for the bugs I see on my server popping here and there. Didnt expect that much users and traffic. {'neg': 0.091, 'neu': 0.75, 'pos': 0.159, 'compound': 0.8823}
I like it! I definitely could use something like this. I am GLAD that you mention &quot;It’s for your eyes only.&quot;. Very important feature. {'neg': 0.0, 'neu': 0.537, 'pos': 0.463, 'compound': 0.9156}
Thanks for making this open-source. It seems to use Homestead for deployments and Homestead apparently needs a VM. Is there a way 

In [20]:
!pip install TextBlob

In [32]:
from textblob import TextBlob
for index,row in max_desc_comments.iterrows():
    sentence = row['text']
    testimonial = TextBlob(str(sentence))
    print("{:-<40} {}".format(sentence, testimonial.sentiment))

Looks neat. As an aside, why is it named after a woman? I&#x27;m genuinely curious. Sentiment(polarity=-0.1, subjectivity=1.0)
Founder here. Here to answer any questions you might have. The site is not perfect, it&#x27;s not mobile optimized, there are probably bugs, there is a gazillion features missing, no APIs but it&#x27;s a labour of love, open-source and I hope it will help people other than me. I want to grow this product but I need to know what you need, people.
Edit: sorry for the bugs I see on my server popping here and there. Didnt expect that much users and traffic. Sentiment(polarity=-0.10416666666666667, subjectivity=0.5375)
I like it! I definitely could use something like this. I am GLAD that you mention &quot;It’s for your eyes only.&quot;. Very important feature. Sentiment(polarity=0.34, subjectivity=0.8333333333333334)
Thanks for making this open-source. It seems to use Homestead for deployments and Homestead apparently needs a VM. Is there a way to make this work wit